# 今さら聞けないGANシリーズ(5)

https://qiita.com/triwave33/items/72c7fceea2c6e48c8c07  
WGAN-gpの実装

## モデル定義
discriminator_with_own_lossモデルの実装  
WGAN-gpでは，正解ラベルと予想結果を付き合わせる形式を用いない．  
binary cross entropyなどではなく，損失関数を独自に定義する．  

実装の参考  
https://github.com/tjwei/GANotebooks/blob/master/wgan2-keras.ipynb

In [ ]:
from gan import GAN

class WGANGP(GAN):
    def build_discriminator_with_own_loss(self):
        # 1. モデルの作成
        
        # generatorの入力
        z = Input(shape=(self.z_dim,))
        
        # discriminatorの入力
        f_img = self.generator(g_input)
        img_shape = (self.img_rows, self.img_cols, self.channels)
        r_img = Input(shape(img_shape))
        
        # 生成データと偽物のデータの比率を決める値
        e_input = K.placeholder(shape=(None, 1, 1, 1))
        
        a_img = Input(shape=(img_shape), \
                     tensor=e_input * img_input + (1-e_input) * g_output)
        
        # discriminatorの出力
        f_out = self.discriminator(f_img)
        r_out = self.discriminator(r_img)
        a_out = self.discriminator(a_img)
        
        
        
        # 2. 損失関数の作成
        
        # original critic loss
        loss_real = K.mean(r_out)
        loss_fake = K.mean(f_out)
        
        # gradient penalty
        grad_mixed = K.gradients(a_out, [a_img])[0]
        norm_grad_mixed = K.sqrt(K.sum(K.square(grad_mixed), axis=[1,2,3]))
        grad_penalty = K.mean(K.square(norm_grad_mixed - 1))
        
        # 最終的な損失関数
        loss = loss_fake - loss_real + GRADIENT_PENALTY_WEIGHT * grad_penalty
        
        
        
        # 3. optimizer をインスタンス化
        training_updates = Adam(lr=1e-4, beta_1=0.5, beta_2=0.9)\
            .get_updates(self.discriminator.trainable_weights, [], loss)
        
        
        
        # 4. 入出力とoptimizerをfunction化
        d_train = K.function([img_input, g_input, e_input], \
                            [loss_real, loss_fake], \
                            training_updates)
        
        return d_trainb
    